# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: Diogo Nobre de Araujo Cintra

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [3]:
%matplotlib inline
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
from numpy import arange
from numpy import percentile
import numpy as np
import os
import re 

___
## Autenticando no  Twitter

* Conta: ***@Ciencia dos dados***

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Capturando os dados do twitter:

Salvando os dados em uma planilha Excel:

___
### Classificando as mensagens na coragem

#### 0 = irrelevante
#### 1 = relevante

In [10]:
TREINAMENTO = pd.read_excel('Correios.xlsx',sheet_name = 'Treinamento')
TESTE = pd.read_excel('Correios.xlsx',sheet_name = 'Teste')
TREINAMENTO

,Treinamento,Classificacao
0,rt @arthurmoledoval: o governo recomendou ao b...,0
1,"rt @casonatogabriel: a amazon, empresa na qual...",0
2,rt @arthurmoledoval: o governo recomendou ao b...,0
3,"@willianlofy hoje usam correios quem quer, com...",0
4,@eduwilbert já acordei puto com os correios kk...,1
5,@filipeazero @folha os correios oferecem servi...,1
6,"rt @iaragb: cpi dos correios - r$3.863.955,35 ...",1
7,o sistema de entrega dos correios é péssimo,1
8,"rt @casonatogabriel: a amazon, empresa na qual...",0
9,rt @arthurmoledoval: o governo recomendou ao b...,0


In [5]:
# retira a pontuação dos textos
def clean(text):
    punctuation = '[!\-.:?;#$%&*_1234567890"]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [9]:
texto = " ".join(TREINAMENTO['Treinamento'])
texto_limpo_1 = clean(texto.lower())
texto_limpo_2 = pd.Series(texto_limpo_1.split())
texto_limpo_3 = []

for e in range(len(texto_limpo_2)):
    if texto_limpo_2[e] != 'rt' and texto_limpo_2[e][0] != '@':     
        texto_limpo_3.append(texto_limpo_2[e])  
        
        
texto_limpo_3

['o',
 'governo',
 'recomendou',
 'ao',
 'bndes',
 'que',
 'os',
 'estudos',
 'sobre',
 'uma',
 'eventual',
 'privatização',
 'dos',
 'correios',
 'fiquem',
 'prontos',
 'em',
 'até',
 'mese…',
 'a',
 'amazon,',
 'empresa',
 'na',
 'qual',
 'queremos',
 'transformar',
 'os',
 'correios,',
 'lança',
 'hoje',
 'no',
 'brasil',
 'o',
 'programa',
 'prime,',
 'que',
 'dá',
 'aos',
 'seus',
 'ass…',
 'o',
 'governo',
 'recomendou',
 'ao',
 'bndes',
 'que',
 'os',
 'estudos',
 'sobre',
 'uma',
 'eventual',
 'privatização',
 'dos',
 'correios',
 'fiquem',
 'prontos',
 'em',
 'até',
 'mese…',
 'hoje',
 'usam',
 'correios',
 'quem',
 'quer,',
 'como',
 '“todos”',
 'sabem,',
 'o',
 'monopólio',
 'é',
 'somente',
 'daquelas',
 'cartinhas',
 'que',
 'ninguém',
 'mais',
 'usa',
 'já',
 'acordei',
 'puto',
 'com',
 'os',
 'correios',
 'kkkkkk',
 'os',
 'correios',
 'oferecem',
 'serviço',
 'postal',
 'a',
 'todo',
 'territorio',
 'nacional',
 'independente',
 'de',
 'retorno',
 'financeiro,',
 'o',


In [7]:
#probabilidade de ser relevante
prev

NameError: name 'prev' is not defined

In [ ]:
#probabilidade de não ser relevante
pnorev

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
#contador das palavras

In [ ]:
#associa um valor de probabilidade
probrev={}
#formato {palavra:valor}
probnorev={}
#formato {palavra:valor}

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
smoth = 1/1000000 #é uma estimativa e deve mudar
# defini probabilidade de ser relevante:
def relevante(tweet):
    t = clean(tweet)
    prob = 1
    for p in t:
        if p in probrev:
            prob = prob*(probrev(p)+smoth)
        else:
            prob = prob*(smoth)
    prob = prev
# defini probabilidade de ser irrelevante:
def relevante(tweet):
    t = clean(tweet)
    prob = 1
    for p in t:
        if p in probnorev:
            prob = prob*(probnorev(p)+smoth)
        else:
            prob = prob*(smoth)
    prob = pnorev

In [ ]:
def compara (tweet):
    r = relevante(tweet)
    nor = irrelevante(tweet)
    return r>nor

In [ ]:
arquivo = 
rev = 0
norev = 0
for txt in arquivo:
    if compara (txt):
        rev +=1
    else:
        norev+=1
print ("a probabilidade de ser relevante é {}".format(rev/arquivo.len()))
print ("a probabilidade de não ser relevante é {}".format(rev/arquivo.len()))
if rev/arquivo.len() < 0.9*prev:
    print ("poucos relevantes")
elif rev/arquivo.len() > 1.1*prev:
    print ("muitos relevantes")
else:
    print ("ok: {} veze o esperado".format (rev/arquivo.len()/prev))


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**